In [4]:
import pandas as pd
import re

In [1]:
s_loc = './data/sample/'
d_sets = ['Indian-Male-Names.csv','Indian-Female-Names.csv']
d_sets

['Indian-Male-Names.csv', 'Indian-Female-Names.csv']

In [2]:
map_info = {'gender': {0:'Male',1:'Female'}, 'race' : {0:'black',1:'hispanic',2:'white',3:'indian'}}
map_info

{'gender': {0: 'Male', 1: 'Female'},
 'race': {0: 'black', 1: 'hispanic', 2: 'white', 3: 'indian'}}

In [5]:
ind_m = pd.read_csv(s_loc+d_sets[0])
ind_f = pd.read_csv(s_loc+d_sets[1])
ind_f.head()

,name,gender,race,length
0,shivani,1,3,7
1,isha,1,3,4
2,smt shyani devi,1,3,15
3,divya,1,3,5
4,mansi,1,3,5


In [6]:
ind_m.head()

,name,gender,race,length
0,barjraj,0,3,7
1,ramdin verma,0,3,12
2,sharat,0,3,6
3,birender,0,3,8
4,amit,0,3,4


In [7]:
ind = ind_m[:6800].append(ind_f)
ind.head()

,name,gender,race,length
0,barjraj,0,3,7
1,ramdin verma,0,3,12
2,sharat,0,3,6
3,birender,0,3,8
4,amit,0,3,4


In [8]:
ind.gender.value_counts()

0    6800
1    5685
Name: gender, dtype: int64

In [9]:
len(ind)

12485

## Actual value

In [10]:
y = ind.gender
len(y)

12485

In [34]:
X = ind.name.values
type(X)

numpy.ndarray

## Test-Train split

In [31]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [36]:
X_train.shape

(9988,)

In [37]:
y_test.shape

(2497,)

## Features

In [13]:
def preprocess(word):
    return re.sub(r'[^a-z.\s]+','',str(word))

In [42]:
#pre_t = list(map(lambda x: preprocess(x),ind.name.values))
names_v = list(X_train)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
vec = CountVectorizer(lowercase=True,token_pattern='.')

In [45]:
name_vec = vec.fit_transform(names_v)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [18]:
name_vec

<12485x27 sparse matrix of type '<class 'numpy.int64'>'
	with 97580 stored elements in Compressed Sparse Row format>

In [168]:
print(vec.vocabulary_)

{'b': 4, 'a': 3, 'r': 20, 'j': 12, 'm': 15, 'd': 6, 'i': 11, 'n': 16, ' ': 1, 'v': 24, 'e': 7, 's': 21, 'h': 10, 't': 22, 'c': 5, 'l': 14, 'k': 13, 'u': 23, 'p': 18, 'g': 9, 'y': 27, 'w': 25, 'o': 17, '.': 2, 'f': 8, 'z': 28, 'x': 26, 'q': 19, '\t': 0}


In [19]:
print(vec.vocabulary_)

{'b': 2, 'a': 1, 'r': 18, 'j': 10, 'm': 13, 'd': 4, 'i': 9, 'n': 14, ' ': 0, 'v': 22, 'e': 5, 's': 19, 'h': 8, 't': 20, 'k': 11, 'u': 21, 'l': 12, 'p': 16, 'g': 7, 'y': 25, 'w': 23, 'o': 15, 'f': 6, 'c': 3, 'z': 26, 'x': 24, 'q': 17}


### Convert name

In [20]:
def name_prep(word):
    #te = re.sub(r'/.*|@.*','',word).strip()
    te = str(word).lower()
    te = re.sub(r'[^a-z.\s].*','',te).strip()
    te = re.sub(r'[^a-z ]+','',te).strip()
    te = re.sub(r' +',' ',te).strip()
    te = re.sub(r'along.*|with.*|and.*',' ',te).strip()
    #te = re.sub(r'\b\w$|\b\w\w$','',te).strip()
    te = re.sub(r'\b\w\b|\b\w\w\b','',te).strip()
    #te = re.sub(r'\bna\b|\bNaN\b','',te).strip()
    if(len(te) < 3):
        return None
    else:
        return te

In [22]:
def conv_input(name,vectorizer):
    pre = name_prep(name)
    return vectorizer.transform([pre])

## Model

In [23]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [24]:
from sklearn.naive_bayes import GaussianNB

In [25]:
naive_b = GaussianNB()

In [26]:
naive_b.fit(name_vec.toarray(),y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [28]:
p = naive_b.predict(conv_input('Narendra Modi',vec).toarray())

In [29]:
map_info['gender'][p[0]]

'Male'